In [1]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [81]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

In [82]:
#!wget https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/rps_data_sample.zip
#!unzip rps_data_sample.zip
dataset_path = './drive/MyDrive/sl_img_data'
#dataset_path = 'rps_data_sample'

In [83]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

./drive/MyDrive/sl_img_data
['ㅖ', 'ㄹ', 'ㄴ', 'ㅓ', 'ㅐ', 'ㅋ', 'ㅎ', 'ㅜ', 'ㅌ', 'ㅚ', 'ㅟ', 'ㄱ', 'ㅡ', 'ㅢ', 'ㅣ', 'ㄲ', 'ㅂ', 'ㅃ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅍ', 'ㅑ', 'ㅔ', 'ㅏ', 'ㅒ', 'ㅊ', 'ㅕ', 'ㄸ', 'ㅗ', 'ㄷ', 'ㅛ', 'ㅅ', 'ㅠ', 'ㅁ', 'ㅆ', 'none']


In [88]:
NUM_EXAMPLES = 37

for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  if len(example_filenames) > 0:
    fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
    for i in range(NUM_EXAMPLES):
      axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
      axs[i].get_xaxis().set_visible(False)
      axs[i].get_yaxis().set_visible(False)
    fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [89]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder


In [91]:
hparams = gesture_recognizer.HParams(export_dir="gi_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_3 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 re_lu_3 (ReLU)              (None, 128)               0         
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 37)                4773      
 out (Dense)                                                     
                                                           

In [92]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

359/359 [==============================] - 2s 2ms/step - loss: 0.2207 - categorical_accuracy: 0.8552
Test loss:0.22066722810268402, Test accuracy:0.8551532030105591


In [93]:
model.export_model()
!ls exported_model


Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/canned_gesture_classifier.tflite
best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


In [95]:
files.download('gi_model/gesture_recognizer.task')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [96]:
#모바일용
hparams = gesture_recognizer.HParams(learning_rate=0.003, export_dir="gi_model_2")
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.2)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_4 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 re_lu_4 (ReLU)              (None, 128)               0         
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 37)                4773      
 out (Dense)                                                     
                                                           

In [13]:
loss, accuracy = model_2.evaluate(test_data)
print(f"Test loss:{loss}, Test accuracy:{accuracy}")

2/2 [==============================] - 1s 20ms/step - loss: 0.0830 - categorical_accuracy: 0.9167
Test loss:0.08297133445739746, Test accuracy:0.9166666865348816


In [97]:
from mediapipe.tasks.python import core
from mediapipe.tasks.python import vision
from mediapipe.tasks import python as mp_tasks # Import the 'python' module with alias

model_path = 'gi_model/gesture_recognizer.task'

# Use mp_tasks.BaseOptions instead of core.BaseOptions
base_options = mp_tasks.BaseOptions(model_asset_path=model_path)

In [102]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a gesture recognizer instance with the image mode:
options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path='gi_model/gesture_recognizer.task'),
    running_mode=VisionRunningMode.IMAGE)

# Perform gesture recognition on the provided single image.
# The gesture recognizer must be created with the image mode.
with GestureRecognizer.create_from_options(options) as recognizer:
    import mediapipe as mp

    # Load the input image from an image file.
    mp_image = mp.Image.create_from_file('4.PNG')

    # Load the input image from a numpy array.
    #mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=numpy_image)
    gesture_recognition_result = recognizer.recognize(mp_image)


In [103]:
print(gesture_recognition_result)

GestureRecognizerResult(gestures=[[Category(index=-1, score=0.6643839478492737, display_name='', category_name='ㄷ')]], handedness=[[Category(index=0, score=0.9948424696922302, display_name='Right', category_name='Right')]], hand_landmarks=[[NormalizedLandmark(x=0.023610353469848633, y=0.5420892238616943, z=2.5922776103470824e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.16839852929115295, y=0.4115868806838989, z=-0.025383533909916878, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.3184232711791992, y=0.43001794815063477, z=-0.04271073266863823, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4169881343841553, y=0.5356968641281128, z=-0.053463224321603775, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4553498327732086, y=0.6227945685386658, z=-0.0612926222383976, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.394438236951828, y=0.4544927775859833, z=-0.08660674840211868, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6112970113754272, y=0.